In [0]:
##Written by Roshan Noronha
##Date: September 3, 2019
##Purpose: The purpose of this notebook is to use transfer learning to improve the accuracy of a convolutional neural network. In this context, any improvement over 79% is considered an improvement.

In [2]:
#link google drive
#the trained model will be saved in google drive to be accessed later
#THIS IS IMPORTANT!
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#get dog and cat pictures from kaggle
#https://www.kaggle.com/general/74235
#command line commands are prefaced with a !

!pip install -q kaggle
from google.colab import files
files.upload()

In [0]:
#create a kaggle directory and move files the kaggle.json file there
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

#change permissions of kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

#check that kaggle datasets show up
!kaggle datasets list

In [0]:
#get dog and cat pictures from kaggle
!kaggle competitions download -c dogs-vs-cats

In [0]:
#unzip training and testing data
!unzip train.zip -d train
!unzip test1.zip -d test

In [0]:
#the training folder has dog and cat images together. These need to be in two seperate folders.
!mkdir train/cats/
!mkdir train/dogs/
!mkdir models/

!find train/train/ | grep "/dog.[0-9]*.jpg" | xargs mv -t train/dogs/
!find train/train/ | grep "/cat.[0-9]*.jpg" | xargs mv -t train/cats/

In [0]:
#train folder is empty so it should be removed
rm -r train/train/

In [0]:
#install gpu-tensorflow and check version
!pip install tensorflow-gpu
import tensorflow as tf
print(tf.__version__)


In [0]:
#import keras and dependancies
#for importing vgg16 use keras.applications NOT keras_applications. Otherwise issues.
import keras
from keras.applications.vgg16 import VGG16
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator, image, img_to_array
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D

In [7]:
#define paths to training and testing folders
train_data = "train/"
test_data = "test/test1/"

#preprocess images
#images should be 224 x 224 as that is the input size of the VGG16 network
img_width, img_height = 600, 600

#rescale pixel values from [0-255] to [0-1]
datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(train_data, target_size= (img_width, img_height), batch_size= 32, class_mode= 'binary')

Found 25000 images belonging to 2 classes.


In [15]:
#show summary of VGG16
#note that the last layer should be popped off here. Keras won't remove the last layer later on.
#the final layer of the created sequential model classifies 1000 different categories
#this layer needs to be removed changed to output just two probabilities
vggmodel = VGG16(include_top = False, weights = "imagenet", input_shape= (img_width, img_height, 3))
print(vggmodel.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 600, 600, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 600, 600, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 600, 600, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 300, 300, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 300, 300, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 300, 300, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 150, 150, 128)     0     

In [0]:
#freeze the pretrained layers
for layer in vggmodel.layers:
  layer.trainable = False


In [17]:
#create a sequential model and add the vgg16 layers to it
model = Sequential()
model.add(vggmodel)

#add 3 more layers on at the end
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))

#dropout is added to prevent overfitting i.e. cannot generalize to new examples
#dropout randomly removes activations which forces the network to use other ones
model.add(Dropout(0.5))

#add the last layer of the model so it outputs a probability for cats and dogs
model.add(Dense(1, activation= "softmax"))

model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 18, 18, 512)       14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 165888)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              169870336 
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 1025      
Total

In [0]:
#binary_crossentropy is the loss function that measures the inaccuracy of the prediction (backpropagation)
#it is used since we only have two categories
#the optimizer tweaks the weights of all the layers to minimize the error (gradient descent)
model.compile(loss = 'binary_crossentropy', optimizer= 'rmsprop', metrics = ['accuracy'])

##parameters for training the model
model.fit_generator(train_gen, steps_per_epoch= 400, nb_epoch = 10)

#save the weights after model has been trained
model.save("/content/drive/My Drive/Colab Notebooks/hotdognotahotdog/transferlearning-dogsandcats/trainedmodels/transferlearningdogsandcats3.h5")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=400, epochs=10)`
  after removing the cwd from sys.path.


Epoch 1/10
400/400 [==============================] - 517s 1s/step - loss: 8.0061 - acc: 0.4978
Epoch 2/10
400/400 [==============================] - 510s 1s/step - loss: 7.9152 - acc: 0.5035
Epoch 3/10
400/400 [==============================] - 511s 1s/step - loss: 8.0023 - acc: 0.4980
Epoch 4/10
400/400 [==============================] - 511s 1s/step - loss: 7.9264 - acc: 0.5028
Epoch 5/10
 53/400 [==>...........................] - ETA: 7:24 - loss: 8.3096 - acc: 0.4788

In [12]:
trainedmodel = load_model("/content/drive/My Drive/Colab Notebooks/hotdognotahotdog/transferlearning-dogsandcats/trainedmodels/transferlearningdogsandcats.h5")

#get test data that has been seperated into dogs and cats
#!kaggle datasets download -d chetankv/dogs-cats-images

#unzip training and testing data
#!unzip dogs-cats-images.zip
#!unzip dog\ vs\ cat.zip

#evaluate the effectiveness of the trained model
test_data = "dataset/test_set/"
test_gen = datagen.flow_from_directory(test_data, target_size= (img_width, img_height), batch_size= 32, class_mode= 'binary')
print(trainedmodel.metrics_names)
print(trainedmodel.evaluate_generator(test_gen))

Found 2000 images belonging to 2 classes.
['loss', 'acc']


ValueError: ignored

In [0]:
#import numpy as np
#import matplotlib.pyplot as plt

#for i in range(20, 30, 1):
  #testImage = image.load_img("test/test1/" + str(i) + ".jpg", target_size= (600, 600))
  #testImage = image.img_to_array(testImage)
  #testImage = np.expand_dims(testImage, axis = 0)

  #prediction = trainedmodel.predict(testImage)

  #plt.imshow(image.load_img("test/test1/" + str(i) + ".jpg", target_size= (600, 600)))
  
  #animal = ""
  #if (int(prediction[0][0]) == 0):
    #animal = str(i) + ": cat"
  #else:
    #animal = str(i) + ": dog"
    
  #plt.xlabel("Predicted Value: " + animal)
  #plt.show()
